### Import libraries

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import joblib # save/load models
import pickle # save/load features

from sklearn.metrics import confusion_matrix # for confusion_matrix

import warnings
warnings.filterwarnings('ignore')

### Functions for experiments

In [ ]:
def plot_confusion_matrix(cm, classes, y_test,
                          title=None,
                          normalize=False,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """ 
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    import itertools
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print('Normalized confusion matrix')
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.xlim(-0.5, len(np.unique(y_test))-0.5)
    plt.ylim(len(np.unique(y_test))-0.5, -0.5)
    plt.tight_layout()

In [ ]:
def load_model(filename_model):
    return joblib.load(filename_model)

In [ ]:
def load_objs(filename_objs):
    with open(filename_objs, 'rb') as f:
        try:
            loaded_objs = pickle.load(f)
            training_features = loaded_objs[0]
            test_features = loaded_objs[1]
            training_target  = loaded_objs[2]
            test_target  = loaded_objs[3]
            X_train = loaded_objs[4]
            X_val = loaded_objs[5]
            y_train = loaded_objs[6]
            y_val = loaded_objs[7]
            class_weight  = loaded_objs[8],
            class_names  = loaded_objs[9]
            n_classes = loaded_objs[10]
        except:
            print("Something went wrong when writing to the file")

    return (training_features, test_features, training_target, 
            test_target,X_train, X_val, y_train, y_val,
            class_weight,class_names,n_classes)

### Read data

In [ ]:
df_users = pd.read_csv("data/methods.csv")
activity_types = pd.read_csv("data/activity_types.csv")

### Predict data by each method for plotting

In [ ]:
# define variables
users = df_users[df_users["method"]=="AL"]
method = users["method"].values[0]
y_pred_test_all = []
test_target_all = []
names = []

for user in users["user_id"].values:
    
    user_id = user
    
    filename_model = "users-models/model_user_"+str(user_id)+".sav"
    filename_objs = "users-models/model_user_"+str(user_id)+".pkl"
    
    # load the model from disk
    mcl = load_model(filename_model)
    
    # load the variables from disk
    (training_features, test_features, training_target, 
     test_target, X_train, X_val, y_train, y_val,
     class_weight,class_names,n_classes) = load_objs(filename_objs)
    
    y_pred_test = mcl.predict(test_features)
    predictions_test = [round(value) for value in y_pred_test]
    
    y_pred_val = mcl.predict(X_val)
    predictions_val = [round(value) for value in y_pred_val]
    
    y_pred_test_all = np.append(y_pred_test_all, y_pred_test)
    test_target_all = np.append(test_target_all, test_target)
    names = np.append(names, class_names) 

In [ ]:
names = np.unique(names).astype(int)
test_target_all = test_target_all.astype(int)
y_pred_test_all = y_pred_test_all.astype(int)

### Plot confusion matrices

In [ ]:
cnf_matrix = confusion_matrix(test_target_all, y_pred_test_all, labels=names)
title = "Confusion matrix of {0} condition \n (normalised to % of total test data):".format(method)
plt.figure()
width = 8.5
height = 8.5
plt.figure(figsize=(width, height))
plot_confusion_matrix(cnf_matrix, classes=names, y_test = test_target_all, title = title, normalize=True)